In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install fasttext
!pip install lazypredict

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 1.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pybind11-2.13.6-py3-none-any.whl.metadata (9.5 kB)
Using cached pybind11-2.13.6-py3-none-any.whl (243 kB)
  Created wheel for fasttext: filename=fasttext-0.9.3-cp310-cp310-linux_x86_64.whl size=4296181 sha256=36e8e8cebc1cf3e69bc620f4f2f43788e42b9d1821c4258ed1b8d96e479cdb73
  Stored in directory: /root/.cache/pip/wheels/0d/a2/00/81db54d3e6a8199b829d58e02cec2ddb20ce3e59fad8d3c92a
Successfully built fasttext


In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, KFold
import fasttext
from lazypredict.Supervised import LazyClassifier
from transformers import BertTokenizer
import gensim
import gensim.downloader
from transformers import AlbertTokenizer

dataset = pd.read_excel("/content/drive/MyDrive/domain/Synthetic User Stories.xlsx")

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [ ]:
target = []
for row in dataset.iterrows():
    target.append(np.where(dataset["Domain"].unique() == row[1]["Domain"])[0][0])
dataset["Target"] = target
dataset["Target"]

,Target
0,0
1,1
2,0
3,1
4,0
...,...
12396,37
12397,36
12398,37
12399,36


In [ ]:
def getTrainSetFastText():
    ft_model = fasttext.load_model("fasttext_model.bin")
    traindata = []
    for msg in dataset['User Story']:
        traindata.append(ft_model.get_sentence_vector(msg))
    traindata = pd.DataFrame(traindata)
    traindata.columns = traindata.columns.astype(str)
    return traindata

def getTrainSetTFIDF():
    countvec = CountVectorizer(max_features=100)
    bow = countvec.fit_transform(dataset['User Story']).toarray()
    tfidfconverter = TfidfTransformer()
    X = tfidfconverter.fit_transform(bow).toarray()
    training_data = pd.DataFrame(X)
    training_data.columns = training_data.columns.astype(str)
    return training_data

def getTrainSetBERT():
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    tokenized_data = tokenizer(dataset['User Story'].tolist(), padding=True, truncation=True, max_length=100)
    traindata = []
    for msg in tokenized_data['input_ids']:
        traindata.append(msg)
    traindata = pd.DataFrame(traindata)
    traindata.columns = traindata.columns.astype(str)
    return traindata

def getTrainSetRoBERTa():
    tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
    tokenized_data = tokenizer(dataset['User Story'].tolist(), padding=True, truncation=True, max_length=100)
    traindata = []
    for msg in tokenized_data['input_ids']:
        traindata.append(msg)
    traindata = pd.DataFrame(traindata)
    traindata.columns = traindata.columns.astype(str)
    return traindata

def getTrainSetALBERT():
    tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')
    tokenized_data = tokenizer(dataset['User Story'].tolist(), padding=True, truncation=True, max_length=100)
    traindata = []
    for msg in tokenized_data['input_ids']:
        traindata.append(msg)
    traindata = pd.DataFrame(traindata)
    traindata.columns = traindata.columns.astype(str)
    return traindata

def getTrainSetDistilBERT():
    tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
    tokenized_data = tokenizer(dataset['User Story'].tolist(), padding=True, truncation=True, max_length=100)
    traindata = []
    for msg in tokenized_data['input_ids']:
        traindata.append(msg)
    traindata = pd.DataFrame(traindata)
    traindata.columns = traindata.columns.astype(str)
    return traindata

def getTrainSetWord2Vec():
    w2v_model = gensim.models.KeyedVectors.load_word2vec_format('word2vec-google-news-300.bin', binary=True)
    traindata = []
    for msg in dataset['User Story']:
        words = msg.split()
        vecs = []
        for word in words:
            if word in w2v_model:
                vecs.append(w2v_model[word][:100])
        if vecs:
            vec_avg = sum(vecs) / len(vecs)
        else:
            vec_avg = [0] * 100
        traindata.append(vec_avg)

    traindata = pd.DataFrame(traindata)
    traindata.columns = traindata.columns.astype(str)
    return traindata

def getTrainSetGlove():
    glove_vectors = gensim.downloader.load('glove-wiki-gigaword-100')
    traindata = []
    for msg in dataset['User Story']:
        words = msg.split()
        vecs = []
        for word in words:
            if word in glove_vectors:
                vecs.append(glove_vectors[word])
        if vecs:
            vec_avg = sum(vecs) / len(vecs)
        else:
            vec_avg = [0] * 100
        traindata.append(vec_avg)

    traindata = pd.DataFrame(traindata)
    traindata.columns = traindata.columns.astype(str)
    return traindata

In [ ]:
result2 = pd.DataFrame(columns=["Fold","Model","Accuracy","F1-Score"], index=np.arange(300))
fold = KFold(n_splits=10, random_state=6666, shuffle=True)
X = getTrainSetALBERT() #Change this to get training set based on word embeddings method.
y = dataset['Target']
counter = 0
foldcounter = 1
for train_index, test_index in fold.split(X, y):
        print("Processing Fold "+ str(foldcounter) + " ...")
        X_train, X_test, y_train, y_test = \
            X[ X.index.isin(train_index)], X[ X.index.isin(test_index)], y[train_index], y[test_index]
        clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
        models,predictions = clf.fit(X_train, X_test, y_train, y_test)
        for model in models[:].iterrows():
            result2.loc[counter]["Fold"] = foldcounter
            result2.loc[counter]["Model"] = model[0]
            result2.loc[counter]["Accuracy"] = round(model[1][0],3)
            result2.loc[counter]["F1-Score"] = round(model[1][3],3)
            counter += 1
        foldcounter += 1
result2

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/760k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

Processing Fold 1 ...


...

Processing Fold 10 ...


 97%|█████████▋| 30/31 [32:30<00:07,  7.59s/it]

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004791 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 12448
[LightGBM] [Info] Number of data points in the train set: 11161, number of used features: 83
[LightGBM] [Info] Start training from score -3.506050
[LightGBM] [Info] Start training from score -3.506050
[LightGBM] [Info] Start training from score -3.521088
[LightGBM] [Info] Start training from score -3.536356
[LightGBM] [Info] Start training from score -3.527167
[LightGBM] [Info] Start training from score -3.536356
[LightGBM] [Info] Start training from score -3.554990
[LightGBM] [Info] Start training from score -3.567608
[LightGBM] [Info] Start training from score -3.491235
[LightGBM] [Info] Start training from score -3.503070
[LightGBM] [Info] Start training from score -3.539437
[LightGBM] [Info] Start training from score -3.52412

100%|██████████| 31/31 [32:45<00:00, 63.41s/it]


,Fold,Model,Accuracy,F1-Score
0,1,BaggingClassifier,0.99,0.99
1,1,XGBClassifier,0.98,0.98
2,1,DecisionTreeClassifier,0.98,0.98
3,1,ExtraTreesClassifier,0.92,0.92
4,1,RandomForestClassifier,0.86,0.85
...,...,...,...,...
295,NaN,NaN,NaN,NaN
296,NaN,NaN,NaN,NaN
297,NaN,NaN,NaN,NaN
298,NaN,NaN,NaN,NaN


In [ ]:
result2.to_excel("albert_domain.xlsx")